In [1]:
!pip install scikit-learn lightning timm

In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
import lightning as L
import torch.nn.functional as F
from timm import create_model
import timm
from torchmetrics.classification import Accuracy, F1Score, AUROC
from torch.utils.data import DataLoader
from clearml import Task
import transformers
from urllib.request import urlopen

cudnn.benchmark = True
plt.ion()   # interactive mode

In [2]:
config = {
    'epochs': 50,
    'optimizer': transformers.Adafactor,
    'optimizer_': 'Adafactor',
    'lr': 1e-03,
    'scheduler': lr_scheduler.StepLR,
    'scheduler_': 'StepLR',
    'scheduler_step_size': 7,
    'gamma': 0.2,
    'batch_size': 32,
    'loss': F.cross_entropy,
    'loss_': 'cross_entropy',
}

In [3]:
from torchvision.transforms import v2

IMG_SIZE=336

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.8, 1.0), ratio=(0.5, 0.6)),
        transforms.RandomHorizontalFlip(),
        v2.RandomRotation(45),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.CenterCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/home/raid_storage/astashkin/hackathon/data/dataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(image_datasets['train'], batch_size=config['batch_size'], shuffle=True, num_workers=32)
dataloaders['val'] = torch.utils.data.DataLoader(image_datasets['val'], batch_size=config['batch_size'], shuffle=False, num_workers=32)

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [4]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, roc_auc_score, roc_curve
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

def imshow(inp, title=None):
    """Display image for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize = (8,10))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) 

def visualize_model(model, num_images=6):
    with torch.no_grad():
        (inputs, labels) = next(iter(dataloaders['val']))
        inputs = inputs[:num_images]
        labels = labels[:num_images]
    
        outputs = model.predict_step((inputs, labels), 0)
        _, preds = torch.max(outputs, 1)
    
        outputs = torchvision.utils.make_grid(inputs).cpu()
        imshow(outputs, title=[class_names[x] for x in preds])

def draw_conf_matrix(model, dataloader):
    y_true = torch.empty(0, dtype=torch.int)
    y_pred = torch.empty(0, dtype=torch.int)
    y_score = torch.empty((len(dataloader.dataset), len(class_names)), dtype=torch.float64)
    
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            outputs = model.predict_step((inputs, labels), i)

            _, preds = torch.max(outputs, 1)
            
            y_true = torch.cat((y_true, labels.cpu()))
            y_pred = torch.cat((y_pred, preds.cpu()))
        
        cf_matrix = confusion_matrix(y_true, y_pred)
        df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [j for j in class_names], columns = [j for j in class_names])
        plt.figure(figsize = (8,6))
        plt.title('Confusion matrix')
        sn.heatmap(df_cm, annot=True)
        plt.show()

In [ ]:
# МОЖЕТ ПАДАТЬ ПО ПРИКОЛУ ИЛИ ВИСНУТЬ

# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])

In [5]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix
from torchmetrics import F1Score, Precision, Recall, Metric, MetricCollection, AUROC

def get_metrics(**kwargs) -> MetricCollection:
    return MetricCollection(
        {
            'accuracy': Accuracy(**kwargs),
            'f1': F1Score(**kwargs),
            'precision': Precision(**kwargs),
            'recall': Recall(**kwargs),
            'auroc': AUROC(**kwargs)   
        }
    )

In [6]:
class LitWrapper(L.LightningModule):
    def __init__(self, model):
        super().__init__()

        self.model = model
        
        metrics = get_metrics(
            num_classes=len(class_names),
            task='multiclass',
            average='macro'
        )

        self.metrics = [i for i in metrics.keys()]
        self._train_metrics = metrics.clone(prefix='train_')
        self._valid_metrics = metrics.clone(prefix='val_')
        self._test_metrics = metrics.clone(prefix='test_')

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        preds = self.model(inputs)
        loss = config['loss'](preds, labels)
        
        _, pred_labels = torch.max(preds, dim=1)
        self.log('train_loss', loss)
        
        for metric in self.metrics:
            if metric != 'auroc':
                self._train_metrics[metric](pred_labels, labels)
            else:
                self._train_metrics[metric](preds, labels)
                
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        preds = self.model(inputs)

        # print('preds', torch.unique(preds), preds.shape)
        # print('unique labels', torch.unique(labels), labels.shape)
        
        loss = config['loss'](preds, labels)
        
        _, pred_labels = torch.max(preds, dim=1)

        # print('pred labels', torch.unique(pred_labels), pred_labels.shape)
        
        self.log('val_loss', loss)
        for metric in self.metrics:
            if metric != 'auroc':
                self._valid_metrics[metric](pred_labels, labels)
            else:
                self._valid_metrics[metric](preds, labels)
                
        return loss

    def predict_step(self, batch, batch_idx):
        inputs, labels = batch
        preds = self.model(inputs)       
        return preds

    def configure_optimizers(self):
        optimizer = config['optimizer'](self.model.parameters(), lr=config['lr'], relative_step=False, warmup_init=False)
        scheduler = config['scheduler'](optimizer, step_size=config['scheduler_step_size'], gamma=config['gamma'])
        return [optimizer], [scheduler]

    def train_dataloader(self):
        return dataloaders['train']

    def val_dataloader(self):
        return dataloaders['val']

    def on_validation_epoch_start(self) -> None:
        self._valid_metrics.reset()

    def on_validation_epoch_end(self) -> None:
        self.log_dict(self._valid_metrics.compute(), on_epoch=True)

    def on_test_epoch_end(self) -> None:
        self.log_dict(self._test_metrics.compute(), on_epoch=True)

*******************************************

        eva02_large_patch14_336

********************************************

In [7]:
img = Image.open(urlopen(
    'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'
))

model = timm.create_model('eva_large_patch14_336.in22k_ft_in22k_in1k', pretrained=True)
movel = model.eval()
model.head = nn.Linear(1024, len(class_names))
movel = model.eval()

data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=True)

model.eval()

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Id

In [ ]:
task = Task.init(project_name="hackathon", task_name="eva02_large_patch14_336")

task.connect(config)

trainer = L.Trainer(max_epochs=config['epochs'], devices=[2])

model = LitWrapper(model)

trainer.fit(model)

task.upload_artifact(name='best_weights', artifact_object=trainer.checkpoint_callback.best_model_path)

ClearML Task: overwriting (reusing) task id=9a0d9b2587d74263bb1b4823fc327f86
2024-04-13 17:22:45,779 - clearml.Repository Detection - WARNING - Password protected Jupyter Notebook server was found! Add `sdk.development.jupyter_server_password=<jupyter_password>` to ~/clearml.conf
2024-04-13 17:22:45,785 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: http://172.22.100.166:8082/projects/d288a854ffdb4d40aa2eba58d94efebb/experiments/9a0d9b2587d74263bb1b4823fc327f86/output/log
2024-04-13 17:22:45,949 - clearml.Task - WARNING - Parameters must be of builtin type (General/optimizer[type], General/scheduler[type], General/loss[function])


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name           | Type              | Params
-----------------------------------------------------
0 | model          | VisionTransformer | 303 M 
1 | _train_metrics | MetricCollection  | 0     
2 | _valid_metrics | MetricCollection  | 0     
3 | _test_metrics  | MetricCollection  | 0     
-----------------------------------------------------
303 M     Trainable params
0         Non-trainable params
303 M     Total params
1,214.087 Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning:

No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score

/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning:

No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score



Training: |          | 0/? [00:00<?, ?it/s]

ClearML results page: http://172.22.100.166:8082/projects/d288a854ffdb4d40aa2eba58d94efebb/experiments/9a0d9b2587d74263bb1b4823fc327f86/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
trainer.validate(model)
draw_conf_matrix(model, dataloaders['val'])

# МОЖЕТ ПАДАТЬ ПО ПРИКОЛУ
# visualize_model(model)